## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [6]:
import os
# Load environment variables
load_dotenv(override=True)

# Get your Gemini API key from the .env file
api_key = os.getenv("GEMINI_API_KEY")

# Initialize OpenAI client for Gemini using Google's OpenAI-compatible endpoint
openai = OpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)

In [7]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [8]:
print(linkedin)

   
Contact
workwithaayush2095@gmail.co
m
www.linkedin.com/in/aayush2095
(LinkedIn)
Top Skills
End-to-End Product Development
Product Requirements Gathering
Feature Prioritization
Certifications
Product Management Basics
Certification
Product Discovery Certification
Aayush Agrawal
Aspiring Product Manager | Bridging Tech & Product Strategy |
Passionate About Delivering Scalable Solutions & Improving User
Experience | Product Focused Engineer
Bengaluru, Karnataka, India
Summary
As a Senior Software Engineer at Subex, I’ve developed and
optimized scalable solutions, handling high-volume telecom data
processing and improving system performance. With expertise in
core Java, and microservices architecture, I have a track record of
improving system uptime, reducing response times, and automating
complex processes.
Alongside my engineering background, I am deeply passionate
about Product Management. I am currently learning the intricacies of
product strategy, user-centered design, and go-to-m

In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Aayush Agrawal"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

"You are acting as Aayush Agrawal. You are answering questions on Aayush Agrawal's website, particularly questions related to Aayush Agrawal's career, background, skills and experience. Your responsibility is to represent Aayush Agrawal for interactions on the website as faithfully as possible. You are given a summary of Aayush Agrawal's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nAs a Senior Software Engineer at Subex, I’ve developed and optimized scalable solutions, handling high-volume telecom data processing and improving system performance. With expertise in core Java, and microservices architecture, I have a track record of improving system uptime, reducing response times, and automating complex processes.\n\nAlongside my engineering background, I am deeply passionate about Product Manage

In [14]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gemini-2.5-flash", 
        messages=messages
    )
    return response.choices[0].message.content


## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [15]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [16]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [17]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [18]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [19]:
import os
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [20]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [22]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "do you hold a patent?"}
]

response = openai.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=messages
)

reply = response.choices[0].message.content


In [23]:
reply

"That's an interesting question! Thank you for asking.\n\nBased on my background and experience, I do not currently hold any patents. My work has primarily focused on developing and optimizing software solutions, leading technical initiatives, and collaborating on product strategy within the telecom domain.\n\nWhile I haven't pursued patents, I'm deeply committed to innovation and creating impactful solutions. My aim is to leverage my technical expertise and growing passion for product management to shape cutting-edge products that solve real-world problems.\n\nIs there anything else I can tell you about my experience or skills?"

In [24]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is good, it is in line with the persona and provides an honest answer to the question, while also redirecting the conversation back to the core skills and experience of the agent.')

In [25]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + (
        "\n\n## Previous answer rejected\n"
        "You just tried to reply, but the quality control rejected your reply.\n"
        f"## Your attempted answer:\n{reply}\n\n"
        f"## Reason for rejection:\n{feedback}\n\n"
    )

    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model="gemini-2.5-flash",
        messages=messages
    )

    return response.choices[0].message.content


In [29]:
def chat(message, history):
    # Update system prompt conditionally based on "patent"
    if "patent" in message.lower():
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - " \
                                 "it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    # Build messages
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]

    # Call Gemini via OpenAI-compatible client
    response = openai.chat.completions.create(
        model="gemini-2.5-pro", 
        messages=messages
    )
    reply = response.choices[0].message.content

    # Evaluate the response
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)

    return reply


In [30]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The agent responded in pig latin which is unprofessional. This is not appropriate for the persona and use case.
Failed evaluation - retrying
The Agent has created a response in pig latin, which is not professional. This is likely to do with the inclusion of 'options' in the turn schema. I have marked this as not acceptable. There response also doesn't quite make sense.
